In [1]:
import pandas as pd

df = pd.read_csv('/Users/jacksb/github-classroom/iSTAREducation/sml-mid-term-assignemnts-jackb-22/data/image_data.csv')
y = df['Label'].values
X = df.drop(columns=['Label']).values

In [2]:
# perform train - test split. (40% as testing and 60% as training)
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split()

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                    test_size = 0.40, 
                                    random_state=1)
X_test.shape

(315, 12288)

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
steps = [
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
]
DT_pipeline = Pipeline(steps)

param_grid = {
    'knn__n_neighbors': [3, 5, 7, 9, 11],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'minkowski'],
    'knn__p': [1, 2] 
}

grid_search = GridSearchCV(
    estimator=DT_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__metric': ['euclidean', 'manhattan', 'minkowski'],
                         'knn__n_neighbors': [3, 5, 7, 9, 11], 'knn__p': [1, 2],
                         'knn__weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=1)

In [5]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

best_model = grid_search.best_estimator_
ypred_train = best_model.predict(X_train)
mat_clf = confusion_matrix(y_train, ypred_train)
report_clf = classification_report(y_train, ypred_train)

print("Confusion Matrix:\n", mat_clf)
print("\nClassification Report:\n", report_clf)

ypred_trainP = best_model.predict_proba(X_train)
auc = roc_auc_score(y_train, ypred_trainP[:, 1])

print("\nROC AUC Score:", auc)

Confusion Matrix:
 [[237   0]
 [  0 235]]

Classification Report:
               precision    recall  f1-score   support

         car       1.00      1.00      1.00       237
       truck       1.00      1.00      1.00       235

    accuracy                           1.00       472
   macro avg       1.00      1.00      1.00       472
weighted avg       1.00      1.00      1.00       472


ROC AUC Score: 1.0


In [6]:
best_model = grid_search.best_estimator_

ypred_test = best_model.predict(X_test)

mat_clf = confusion_matrix(y_test, ypred_test)
report_clf = classification_report(y_test, ypred_test)

print("Confusion Matrix:\n", mat_clf)
print("\nClassification Report:\n", report_clf)

ypred_testP = best_model.predict_proba(X_test)
auc = roc_auc_score(y_test, ypred_testP[:, 1])

print("\nROC AUC Score:", auc)

Confusion Matrix:
 [[83 71]
 [66 95]]

Classification Report:
               precision    recall  f1-score   support

         car       0.56      0.54      0.55       154
       truck       0.57      0.59      0.58       161

    accuracy                           0.57       315
   macro avg       0.56      0.56      0.56       315
weighted avg       0.56      0.57      0.56       315


ROC AUC Score: 0.6413245139953214
